In [15]:
import numpy as np
from pathlib import Path
import sys
import pandas as pd

import matplotlib.pyplot as plt
import networkx as nx

sys.path.append('../../../bitome-kb/')

from bitome.core import Bitome

In [16]:
bitome = Bitome.init_from_file('../../bitome-kb/bitome.pkl')

In [17]:
mutation_df = pd.read_pickle('../../../data/filtered_K12_mut_df.pkl')
mutation_df.head()

,exp,ale,flask,isolate,tech_rep,presence,Reference Seq,Position,Mutation Type,Details,Sequence Change,Gene,coding,organism
43,42C,1,124,1,1,1.0,NC_000913,702352,DEL,coding (380‑400/1149 nt),Δ21 bp,nagA,True,Escherichia coli
49,42C,1,124,1,1,1.0,NC_000913,1308318,SNP,R110G (CGT→GGT),G→C,clsA,True,Escherichia coli
25,42C,1,124,1,1,1.0,NC_000913,2173364,DEL,intergenic (‑2/+1),Δ1 bp,gatC/gatC,False,Escherichia coli
159,42C,1,124,1,1,1.0,NC_000913,3815859,DEL,,Δ82 bp,rph,True,Escherichia coli
161,42C,1,124,1,1,1.0,NC_000913,4187550,SNP,A734V (GCG→GTG),C→T,rpoC,True,Escherichia coli


In [18]:
snp_df = mutation_df[mutation_df['Mutation Type'] == 'SNP']
snp_df.head()

,exp,ale,flask,isolate,tech_rep,presence,Reference Seq,Position,Mutation Type,Details,Sequence Change,Gene,coding,organism
49,42C,1,124,1,1,1.0,NC_000913,1308318,SNP,R110G (CGT→GGT),G→C,clsA,True,Escherichia coli
161,42C,1,124,1,1,1.0,NC_000913,4187550,SNP,A734V (GCG→GTG),C→T,rpoC,True,Escherichia coli
13,42C,1,124,1,1,1.0,NC_000913,4400313,SNP,D9A (GAT→GCT),A→C,hfq,True,Escherichia coli
18,42C,2,163,1,1,1.0,NC_000913,149214,SNP,A130T (GCC→ACC),C→T,panB,True,Escherichia coli
128,42C,2,163,1,1,1.0,NC_000913,219624,SNP,C124C (TGT→TGC),A→G,rcsF,True,Escherichia coli


In [19]:
tu_promoters = [tu.promoter for tu in bitome.transcription_units if tu.promoter is not None]
tu_promoters[0].location
end_location = max(tu_promoters[0].location)
start_location = min(tu_promoters[0].location)

In [20]:
snp_df[(snp_df['Position'] > min(tu_promoters[0].location)) & (snp_df['Position'] < max(tu_promoters[0].location))]

,exp,ale,flask,isolate,tech_rep,presence,Reference Seq,Position,Mutation Type,Details,Sequence Change,Gene,coding,organism


In [21]:
dfs_with_overlap = []
for promoter in tu_promoters:
    snp_overlap = snp_df[(snp_df['Position'] > min(promoter.location)) & (snp_df['Position'] < max(promoter.location))]
    if not snp_overlap.empty:
        dfs_with_overlap.append(snp_overlap)
mutations_in_promoters = pd.concat(dfs_with_overlap)
print(mutations_in_promoters)

                                                   exp  ale  flask  isolate  \
109                           TOL_hexamethylenediamine    4     50        3   
236                                       EEP_ndh_nuoB    6     34        0   
0                                         EEP_ndh_nuoB    6     34        0   
256                                       EEP_ndh_nuoB    6     34        0   
208                                       EEP_ndh_nuoB    6     34        0   
..                                                 ...  ...    ...      ...   
289                                    EEPcydBappCcyoB    2     21        0   
36                                     EEPcydBappCcyoB    2     21        0   
453  BOP27-M9-NC_000913_3gb-log-37-Glucose2-NH4Cl1-...    7    422       53   
260                                                GLV    2     57        1   
340                                           HOT_dnaQ   12     56        1   

     tech_rep  presence Reference Seq  Position Mut

In [22]:
# download as csv

mutations_in_promoters.to_csv('SNP_in_promoters_overlap.csv')